In [10]:
from aerolib import *
from xfoillib import *
import pandas as pd
import numpy as np

In [11]:
def isNaN(num):
    return num != num

In [12]:
class InsufficientInputData(Exception):
    def __init__(self, text):
        self.txt = text

In [13]:
class Foil():
    def __init__(self, name):
        pass
    def get_geometry(self):
        pass
    def get_filename(self):
        pass
    

In [14]:
class Sculptor():
    def __init__(self, performance_file_name, params_file_name, settings_file_name, m):
        self.params_file_name = params_file_name
        self.geom_file_name = "GEOM.csv"
        self.aero_file_name = "AERO.csv"
        df = pd.read_csv(performance_file_name, header=None)[1:]
        self.performance = dict(zip(df[0], [float(x) for x in df[1]]))
        self.preprocess_performance()

        df = pd.read_csv(settings_file_name, header=None)[1:]
        self.settings = dict(zip(df[0], [x for x in df[1]]))
        self.preprocess_settings()

        df=pd.read_csv(params_file_name, index_col=False)
        self.params = dict(filter(lambda x: x[0][0]!="#", dict(zip(df["NAME"], [float(x) for x in df["VALUE"]])).items()))
        self.preprocess_params()

        self.geom = {}
        self.aero = {"cruise":np.nan}
        self.tom = m

    def preprocess_params(self):
        critical_params = ['CL_take_off', 'eta_prop', 'A_aft', 
                           'B_keel', 'l_stab']
        for name in critical_params:
            if isNaN(self.params.get(name, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')

    def preprocess_performance(self):
        critical_performance = ['cruise_speed', 'take_off_speed', 'flight_time']
        for name in critical_performance:
            if isNaN(self.performance.get(name, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')
    
    def preprocess_settings(self):
        if isNaN(self.settings.get('g', np.nan)):
            self.settings['g'] = 9.81
            
        if isNaN(self.settings.get('density', np.nan)):
            self.settings['density'] = 1.22

        if isNaN(self.settings.get('wire_scale_coef', np.nan)):
            self.performance['wire_scale_coef'] = 1.3
        
        critical_settings = ['dyn_viscosity', 'density', 'g', 'Re', 'M', 
                             'xfoil_max_it', 'ncr', 'alpha_min', 'alpha_max', 'alpha_step', 
                             'ar_min', 'ar_max', 'ar_delta', 'osvald_coef', 
                             'wire_scale_coef', 'XFoil_path', 'foil1_name', 'work_dir']
        for name in critical_settings:
            if isNaN(self.settings.get(name, np.nan)):
                raise InsufficientInputData(f'obligatory parametr {name} not found')

    def calculate_geometry(self):        
        self.geom["wing_area"] = wing_area(self.tom, float(self.settings["g"]),
                                           float(self.settings["density"]), 
                                           float(self.performance["take_off_speed"]), 
                                           float(self.params["CL_take_off"]))
        self.CL_cr = CL_cruise(self.tom, self.performance["cruise_speed"],
                               float(self.geom["wing_area"]), float(self.settings["g"]), 
                               float(self.settings["density"]))
        ar_step_number = int(abs(float(self.settings["ar_max"]) - float(self.settings["ar_min"])) // float(self.settings["ar_delta"]))
        
        ar_range = np.linspace(float(self.settings["ar_min"]), float(self.settings["ar_max"]), ar_step_number)
        
        self.geom["AR"] = AR_selector(ar_range, self.geom, 
                                      self.settings, self.performance, self.tom)
        self.aero["cruise"] = K_V_solver(self.geom, self.settings, 
                                         self.performance["cruise_speed"], 
                                         self.geom["AR"], self.tom)
        self.geom["ba"] = ba(self.geom["AR"], self.geom["wing_area"])
        self.geom["wingspan"] = wingspan(self.geom["AR"], self.geom["wing_area"])
        self.geom["foil1_perimeter"] = calc_foil_perimeter(self.settings["work_dir"]+self.settings['foil1_name'] + '.dat')
        self.geom["aft_area"] = aft_area(float(self.params["A_aft"]), 
                                         self.geom["wing_area"], self.geom["ba"], 
                                         float(self.params["l_stab"]))
        self.geom["keel_area"] = keel_area(float(self.params["B_keel"]), 
                                           self.geom["wing_area"], self.geom["wingspan"], 
                                           float(self.params["l_stab"]))
        self.geom["V_dihedral"] = gamma(self.geom["keel_area"], self.geom["aft_area"])
        self.geom["Vtail_area"] = stab_area(self.geom["aft_area"], self.geom["V_dihedral"])
        self.geom["P_cruise"] = P_cruise(self.tom, self.aero["cruise"].K[0], 
                                        float(self.params["eta_prop"]), float(self.settings["g"]))
        self.geom["wire_length"] = wire_length(self.geom["wingspan"], float(self.params["l_stab"]), 
                                              float(self.settings["wire_scale_coef"]))

        self.geom["V_dihedral"] = math.degrees(gamma(self.geom["keel_area"], self.geom["aft_area"]))
        return 0
        
    def update_m(self, new_m):
        self.tom = new_m
    
    def write_geom(self):
        with open(self.geom_file_name, 'w') as f:
            f.write("%s, %s\n" % ("NAME", "VALUE"))
            for key in self.geom.keys():
                f.write("%s, %s\n" % (key, self.geom[key]))
    
    def write_aero(self):
        with open(self.aero_file_name, 'w') as f:
            f.write("%s, %s\n" % ("NAME", "VALUE"))
            for key in self.aero.keys():
                f.write("%s, %s\n" % (key, self.aero[key]))
    
    def write_info(self):
        self.write_geom()
        self.write_aero()
        
        

    def get_data_to_weigh(self):
        return [self.params_file_name, self.geom_file_name]

In [15]:
'''  
Ожидаемый формат params-csv-файла
NAME  INDEX  VALUE LINKED
0   m1    1      1            NaN
1   n1    1     10            NaN
2   m2    2      2            a1;a2
3   n2    2    100            NaN
4   m3    NaN      3          NaN
5    M    NaN   1000          a3;1/cy

Ожидаемый формат geomerty-csv-файла
NAME      VALUE
wing_area 1
ba        1
wingspan  1
AR        1

Подсчёт массы ЛА осуществляется по правилам:
 1) в ходе парсинга двух файлов (params_file, geometry_file) 
    формируется набор слагаемых, которые в сумме образуют массу ЛА
 2) все параметры из params_file, имеющие одинаковые индексы, будут перемножены, образуя слагаемое
 3) если у параметра из params_file в поле LINKED указана связанная величина 
    (или набор связанных величин) из geometry_file, 
    то этот параметр будет перемножен с ней, образуя слагаемое
    (в случае набора, слагаемым станет произведение всех величин из этого набора, умноженное на сам параметр) 
 4) если у величин указан и индекс, и связанная величина, то слагаемым станет 
    произведение всех параметров с одинаковыми индексами и всех у них указанных связанных величин
 5) в случае, если у параметра нет индекса, им становится имя параметра
 6) параметр, у которого не указан индекс и не указана связанная величина, не будет учтён при подсчёте массы
'''

def weigh(params_file_name, geometry_file_name, ask_about_no_value=True, show_keys_in_logs=False):
    geom_df = pd.read_csv(geometry_file_name, header=None).loc[1:] #удаление заголовка из DF
    geom = dict(zip(geom_df[0], geom_df[1])) #создание словаря {Cy: <val>, ...}

    #создание таблицы с колонками вида 
    #[<colomn_num>, <NAME>, <INDEX>, <VALUE>, <LINKED>]
    params = pd.read_csv(params_file_name, index_col=False) 
    params = pd.DataFrame(filter(lambda x: x["NAME"][0] != "#", [params.iloc[i] for i in range(len(params.axes[0]))])).T
    
    #создание словаря 
    #{<index>: <список значений всех параметров и связанных величин, которые в произведении образуют слагаемое>}
    multiply_dict = dict()
    multiply_dict_names = dict()
    TOM = 0
    for var in params: #итерация по колонкам; в var попадает номер колонки (colomn_num)
        params[var]["VALUE"] = float(params[var]["VALUE"])
        if isNaN(params[var]["VALUE"]):
            if ask_about_no_value:
                ans = input(f"\nУ ПАРАМЕТРА {params[var]['NAME']} нет значения - пропустить (y/n)?")
            else:
                ans = "y"
            if ans == "y":
                continue
            raise InsufficientInputData(f"no value for variable '{params[var]['NAME']}'")

        if isNaN(params[var]["INDEX"]) and isNaN(params[var]["LINKED"]):
            continue
        if not isNaN(params[var]["INDEX"]):
            index = params[var]["INDEX"]
        
        else:
            index = params[var]['NAME']

        multiply_dict[index] = multiply_dict.get(index, [])
        multiply_dict_names[index] = multiply_dict_names.get(index, [])

        #сборка слагаемого:
        #добавление параметра к тем, у которых такой же индекс
        multiply_dict[index].append(params[var]["VALUE"])
        multiply_dict_names[index].append(params[var]["NAME"])

        #добавление связанных величин
        if not isNaN(params[var]["LINKED"]):
            linked_list = "".join(params[var]["LINKED"].split()).split(";")
            try:
                linked = list(map(lambda conj: 1/float(geom[conj[2:]]) if conj[0:2] == "1/" else float(geom[conj]), linked_list))
            except KeyError as e:
                print(e)
                raise InsufficientInputData(f"for variable '{params[var]['NAME']}', '{params[var]['LINKED']}' "+\
                                                f"is specified as linked, but it is no '{params[var]['LINKED']}'"+\
                                                f" in the geom-file")
            for conj, conj_name in zip(linked, linked_list):
                multiply_dict[index].append(conj)
                multiply_dict_names[index].append(conj_name)

    #вычисление и суммирование слагаемых
    for item in multiply_dict.items():
        key, vals = item #в vals содержатся все компоненты, в произведении образующие слагаемое
        TOM += math.prod(vals)
        if show_keys_in_logs:
            print(f"[key={key}]\n + ", " * ".join(multiply_dict_names[key]) , f": {TOM}\n", sep="")
        else:
            print(" + ", " * ".join(multiply_dict_names[key]) , f": {TOM}", sep="")
    
    print(f"------------------------\nИтоговая масса: {TOM}\n------------------------\n\n")
    return TOM

In [19]:
ask_about_no_value = 0
show_keys_in_logs = 0
weigh("PARAMS.csv", "GEOM.csv", ask_about_no_value, show_keys_in_logs)

 + l_stab * tube6X5: 0.0078
 + mVTX * nVTX: 0.0318
 + mCrossfire * nCrossfire: 0.0368
 + mRunc_s4 * nRunc_s4: 0.0468
 + mReg * nReg: 0.0718
 + mAT2308 * nAT2308: 0.1298
 + mLiIon18650 * nLiIon18650: 0.27080000000000004
 + mLiIon21700 * nLiIon21700: 0.27080000000000004
 + specific_wing * foil1_perimeter * ba * wingspan * skin_thickness: 0.3332881204924304
 + specific_aft * Vtail_area: 0.36924320983387665
 + mfus * nfus: 0.5192432098338766
 + mMG90 * nMG90: 0.5192432098338766
 + mES9051 * nES9051: 0.5432432098338766
 + specific_3wire * wire_length: 0.5622687422069358
 + mPixhawk * nPixhawk: 0.6102687422069358
 + nPito: 0.6102687422069358
------------------------
Итоговая масса: 0.6102687422069358
------------------------




0.6102687422069358

In [8]:
'''
Модуль расчёта геометрических характеристик потребляет на вход: 
dataframe PERFORMANCE с потребными эксплуатационными характеристиками
dataframe PARAMS с параметрами электронных компонент, конструкционных материалов, аккумуляторов и т.д.
пути до файлов селиговского формата с профилем крыла wing_foil и с профилем оперения aft_foil.
TOM - значение взлётной массы в начальном приближении.

PERFORMANCE включает параметры take_off_speed, cruise_speed, flight_time, payload

PARAMS включает параметры m_FPV, m_powerplant, m_flight_control, m_fus, m_servo1, m_servo2,
line_dens_wire, line_dens_tube1, line_dens_tube2, line_dens_tube3,
area_dens_LWPLA
energy_dens_bat
number_servo1, number_servo2
l_stab
'''

# Все единицы в СИ
def inner_iteration():
    performance_file_name = "PERFORMANCE.csv"#input("имя файла performances: ")
    params_file_name = "PARAMS.csv" #input("имя файла params: ")
    settings_file_name = "SETTINGS.csv" #input("имя файла settings: ") 
    tom = 1 #input("нулевое приближение влётной массы: ")
    max_iter = 10 #input("максимальное число итераций: ")
    
    df = pd.read_csv(settings_file_name, header=None)
    tom_eps = float(dict(zip(df[0], df[1]))["tom_eps"])
    
    sculptor = Sculptor(performance_file_name, params_file_name, settings_file_name, tom)
    i = 1
    ask_about_no_value = True
    while True:
        sculptor.calculate_geometry()
        sculptor.write_info()
        pf, gf = sculptor.get_data_to_weigh()

        print(f"итерация {i}")
        new_tom = weigh(pf, gf, ask_about_no_value)
        ask_about_no_value = False
        if abs(tom - new_tom) > tom_eps:
            tom = new_tom
            sculptor.update_m(new_tom)
        else:
            print(f"сошлось на итерации: {i}")
            print("информация сохранена в файл с геометрией")
            return new_tom
        
        if i == max_iter:
            print(f"прошло {i} итераций, но расчёт всё ещё не завершён.")
            flag = 2
            while flag not in ['0', '1']:
                flag = input(f"введите 1, чтобы произвести ещё {max_iter} операций, иначе 0: ")
            if flag:
                i = 0
            else:
                print("последняя итерация геометрии сохранена")
                break
            
        i+=1
    

In [27]:
m = inner_iteration()

итерация 1



У ПАРАМЕТРА capacity_LiIon18650 нет значения - пропустить (y/n)? y

У ПАРАМЕТРА mPito нет значения - пропустить (y/n)? y

У ПАРАМЕТРА mBaro нет значения - пропустить (y/n)? y

У ПАРАМЕТРА nBaro нет значения - пропустить (y/n)? y


 + l_stab * tube6X5: 0.0078
 + mVTX * nVTX: 0.0318
 + mCrossfire * nCrossfire: 0.0368
 + mRunc_s4 * nRunc_s4: 0.0468
 + mReg * nReg: 0.0718
 + mAT2308 * nAT2308: 0.1298
 + mLiIon18650 * nLiIon18650: 0.27080000000000004
 + mLiIon21700 * nLiIon21700: 0.27080000000000004
 + tube5X4 * ltube5X4: 0.28180000000000005
 + Density_LWPLA * foil1_perimeter * ba * wingspan * skin_thickness: 0.3838095550053561
 + spar1_height * wingspan * spar1_thickness * spar1_density: 0.433984229402104
 + spar2_height * wingspan * spar2_thickness * spar2_density: 0.48711035523395474
 + spar3_height * wingspan * spar3_thickness * spar3_density: 0.5107219667147773
 + spar4_height * wingspan * spar4_thickness * spar4_density: 0.5343335781955999
 + specific_aft * Vtail_area: 0.6067844031017522
 + mfus * nfus: 0.7567844031017522
 + mMG90 * nMG90: 0.7567844031017522
 + mES9051 * nES9051: 0.7807844031017522
 + specific_3wire * wire_length: 0.8014458174003932
 + mPixhawk * nPixhawk: 0.8494458174003933
 + nPito: 0.8494458

In [26]:
XFoil_path = "/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/"
work_path = "/home/crucian/Desktop/aerokittes/Yura_s/OptFALT/"
XFoil_command_CL(16, 7, 0.15, 0.8, 1.5e-5, 0, 9, 1000, XFoil_path, work_path, "SD7037")

0

In [27]:
XFoil_run(work_path, XFoil_path)


  XFOIL Version 6.99
  Copyright (C) 2000   Mark Drela, Harold Youngren

  This software comes with ABSOLUTELY NO WARRANTY,
    subject to the GNU General Public License.

  Caveat computor

 File  xfoil.def  not found

   QUIT    Exit program

  .OPER    Direct operating point(s)
  .MDES    Complex mapping design routine
  .QDES    Surface speed design routine
  .GDES    Geometry design routine

   SAVE f  Write airfoil to labeled coordinate file
   PSAV f  Write airfoil to plain coordinate file
   ISAV f  Write airfoil to ISES coordinate file
   MSAV f  Write airfoil to MSES coordinate file
   REVE    Reverse written-airfoil node ordering
   DELI i  Change written-airfoil file delimiters

   LOAD f  Read buffer airfoil from coordinate file
   NACA i  Set NACA 4,5-digit airfoil and buffer airfoil
   INTE    Set buffer airfoil by interpolating two airfoils
   NORM    Buffer airfoil normalization toggle
   HALF    Halve the number of points in buffer airfoil
   XYCM rr Change CM refere

sh: 1: del: not found


0

In [18]:
XFoil_read(work_path)

sh: 1: del: not found


[-0.283, 0.335, 0.01165]